In [1]:
import torch

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 9)

In [2]:
# %pip install triton bitsandbytes xformers trl peft
# %pip install --upgrade transformers accelerate

In [3]:
# %%capture
# # Colab에서 torch 2.2.1을 사용하고 있으므로, 패키지 충돌을 방지하기 위해 별도로 설치해야 합니다.
# %pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# if major_version >= 8:
#     # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용하세요.
#     %pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
#     %pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass


In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/pervinco/Upstage_Ai_Lab/project/notebooks/llama3_runs/Llama-3-Open-Ko-8B-DialogueSum",
    # model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # 모델 이름을 설정합니다.
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.65 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


FastLanguageModel을 사용하여 특정 모듈에 대한 성능 향상 기법을 적용한 모델을 구성합니다.

- `FastLanguageModel.get_peft_model` 함수를 호출하여 모델을 초기화하고, 성능 향상을 위한 여러 파라미터를 설정합니다.
- `r` 파라미터를 통해 성능 향상 기법의 강도를 조절합니다. 권장 값으로는 8, 16, 32, 64, 128 등이 있습니다.
- `target_modules` 리스트에는 성능 향상을 적용할 모델의 모듈 이름들이 포함됩니다.
- `lora_alpha`와 `lora_dropout`을 설정하여 LoRA(Low-Rank Adaptation) 기법의 세부 파라미터를 조정합니다.
- `bias` 옵션을 통해 모델의 바이어스 사용 여부를 설정할 수 있으며, 최적화를 위해 "none"으로 설정하는 것이 권장됩니다.
- `use_gradient_checkpointing` 옵션을 "unsloth"로 설정하여 VRAM 사용량을 줄이고, 더 큰 배치 크기로 학습할 수 있도록 합니다.
- `use_rslora` 옵션을 통해 Rank Stabilized LoRA를 사용할지 여부를 결정합니다.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=16,  # LoRA 알파 값을 설정합니다.
    lora_dropout=0.1,  # 드롭아웃을 지원합니다. 0.05
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=123,  # 난수 상태를 설정합니다.
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

Unsloth: Already have LoRA adapters! We shall skip this step.


`load_dataset` 함수를 사용하여 특정 데이터셋을 로드하고, 이를 특정 형식으로 포매팅하는 과정을 설명합니다.

- `load_dataset` 함수로 "teddylee777/QA-Dataset-mini" 데이터셋을 "train" 분할로 로드합니다.
- 데이터셋의 각 예제에 대해 `formatting_prompts_func` 함수를 적용하여 포매팅을 수행합니다.
  - 이 함수는 "instruction"과 "output" 필드를 사용하여 주어진 포맷에 맞게 텍스트를 재구성합니다.
  - 재구성된 텍스트는 `alpaca_prompt` 포맷을 따르며, 각 항목의 끝에는 `EOS_TOKEN`을 추가하여 생성이 종료되도록 합니다.
- 최종적으로, 포매팅된 텍스트는 "text" 키를 가진 딕셔너리 형태로 반환됩니다.
- 이 과정을 통해, AI 모델이 처리하기 적합한 형태로 데이터를 전처리하는 방법을 보여줍니다.


In [6]:
import pandas as pd
from datasets import Dataset

def formatting_prompts_func(examples):
    dialogues = examples["dialogue"]
    summaries = examples["summary"]
    texts = []
    for dialogue, summary in zip(dialogues, summaries):
        # dialogue와 summary를 Alpaca 포맷으로 변환하고 EOS_TOKEN을 추가합니다.
        text = alpaca_prompt.format(dialogue, summary) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

train_data = pd.read_csv("../dataset/cleaned_train.csv")
valid_data = pd.read_csv("../dataset/cleaned_dev.csv")

gen_df = pd.read_csv("../dataset/generated_train.csv")
gen_df = gen_df.dropna(subset=['dialogue', 'summary'])

trans_train = pd.read_csv("../dataset/processed_translated_train.csv")

train_data = pd.concat([train_data])

train_data = train_data[['dialogue', 'summary']]
valid_data = valid_data[['dialogue', 'summary']]

EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """
The following is a conversation between multiple people. Summarize the main points of the conversation.

### Conversation:
{}

### Summary:
{}"""

train_dataset = Dataset.from_pandas(train_data)
valid_dataset = Dataset.from_pandas(valid_data)

train_dataset = train_dataset.map(formatting_prompts_func, batched=True,)
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True,)

Map:   0%|          | 0/12457 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

### 모델 훈련하기

이제 Huggingface TRL의 `SFTTrainer`를 사용해 봅시다!

- 참고 문서: [TRL SFT 문서](https://huggingface.co/docs/trl/sft_trainer)


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=train_dataset,  # 학습 데이터셋
    eval_dataset=valid_dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=8,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=-1,  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="epoch",  # 에폭 단위로 평가
        save_strategy="epoch",  # 에폭 단위로 모델 저장
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="./unsloth",  # 출력 디렉토리
        load_best_model_at_end=True  # 최적 모델을 마지막에 로드
    ),
)


/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/12457 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/499 [00:00<?, ? examples/s]

- GPU의 현재 메모리 상태를 확인합니다.
- `torch.cuda.get_device_properties(0)`를 사용하여 첫 번째 GPU의 속성을 조회합니다.
- `torch.cuda.max_memory_reserved()`를 통해 현재 예약된 최대 메모리를 GB 단위로 계산합니다.
- GPU의 총 메모리 크기를 GB 단위로 계산합니다.
- GPU 이름과 최대 메모리 크기, 현재 예약된 메모리 크기를 출력합니다.


In [8]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.65 GB.
6.785 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12,457 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 2,334
 "-____-"     Number of trainable parameters = 167,772,160


Epoch,Training Loss,Validation Loss
0,1.066000,1.313691
1,0.911900,1.321888
2,0.717200,1.391504


PyTorch를 사용하여 모델 훈련 시 메모리 사용량과 훈련 시간을 계산하고 출력하는 코드입니다.

- `torch.cuda.max_memory_reserved()`를 사용하여 훈련 중에 예약된 최대 GPU 메모리를 계산합니다.
- 훈련 시작 시점의 GPU 메모리 사용량과 비교하여 LoRA(Low-Rank Adaptation)를 위해 사용된 추가 메모리 양을 계산합니다.
- 전체 GPU 메모리 대비 사용된 메모리의 비율을 계산합니다.
- 훈련에 소요된 총 시간을 초와 분 단위로 출력합니다.
- 예약된 최대 메모리, LoRA를 위해 사용된 메모리, 그리고 이들이 전체 GPU 메모리 대비 차지하는 비율을 출력합니다.


In [10]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

7585.1162 seconds used for training.
126.42 minutes used for training.
Peak reserved memory = 22.393 GB.
Peak reserved memory for training = 15.608 GB.
Peak reserved memory % of max memory = 94.685 %.
Peak reserved memory for training % of max memory = 65.996 %.


### 추론

모델을 실행해 봅시다! 지시사항과 입력값을 변경할 수 있으며, 출력값은 비워두세요!


`TextStreamer`를 사용하여 연속적인 추론을 수행할 수도 있습니다. 이를 통해 전체를 기다리는 대신 토큰별로 생성 결과를 확인할 수 있습니다.


- `FastLanguageModel.for_inference(model)`을 호출하여 모델의 추론 속도를 2배 향상시킵니다.
- `tokenizer`를 사용하여 특정 포맷의 프롬프트를 토큰화하고, 이를 CUDA 기반의 텐서로 변환합니다. 이 과정에서 피보나치 수열을 계속하는 지시문, 입력값, 그리고 출력값을 위한 빈 공간을 포함합니다.
- `TextStreamer` 객체를 `tokenizer`와 함께 초기화하여 텍스트 스트리밍 기능을 활성화합니다.
- `model.generate` 함수를 호출하여 주어진 입력에 대한 텍스트 생성을 수행합니다. 이때, 최대 128개의 새로운 토큰을 생성할 수 있도록 설정하고, `TextStreamer`를 사용하여 결과를 스트리밍합니다.


`StoppingCriteria`와 `StoppingCriteriaList`를 사용하여 특정 토큰에서 생성을 중단하는 방법을 구현합니다.

- `StopOnToken` 클래스는 `StoppingCriteria`를 상속받아, 생성 중 특정 토큰(`stop_token_id`)이 나타나면 생성을 중단하도록 합니다.
- `stop_token` 변수에 중단할 토큰을 문자열로 지정합니다.
- `tokenizer.encode` 메소드를 사용하여 `stop_token`을 해당 언어 모델의 토큰 ID로 변환합니다.
- `StoppingCriteriaList`에 `StopOnToken` 인스턴스를 포함시켜, 생성 과정에서 이를 중단 조건으로 사용합니다.


In [11]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [12]:
from transformers import TextStreamer

test_data = pd.read_csv('../dataset/test.csv')
FastLanguageModel.for_inference(model)

result_data = []
text_streamer = TextStreamer(tokenizer)

for idx, row in test_data.iterrows():
    print(idx)
    dialogue = row['dialogue']

    prompt = alpaca_prompt.format(dialogue, "")
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    generated_summary = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=4096,  # 최대 생성 토큰 수 설정
        stopping_criteria=stopping_criteria  # 생성 종료 기준
    )

    summary_text = tokenizer.decode(generated_summary[0], skip_special_tokens=True)

    # 불필요한 토큰 제거 후 요약 부분만 추출
    summary_text = summary_text.split("### Summary:")[-1].strip()

    result_data.append({"fname": row['fname'], "summary": summary_text})

result_df = pd.DataFrame(result_data)
print(result_df)
result_df.to_csv('./unsloth/prediction.csv', index=False)


0
<|begin_of_text|>
The following is a conversation between multiple people. Summarize the main points of the conversation.

### Conversation:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.

### Summary:
더슨 씨는 실장에게 메시지를 받아쓰도록

In [13]:
# summary 컬럼에서 ### Summary: 이후 <|end_of_text|> 이전까지만 추출
result_df['summary'] = result_df['summary'].apply(
    lambda x: x.split("### Summary:")[-1].split("<|end_of_text|>")[0].strip()
)

print(result_df)
result_df.to_csv("./unsloth/prediction.csv", index=False)

        fname                                            summary
0      test_0  더슨 씨는 실장에게 메시지를 받아쓰도록 요청하고, 실장은 모든 직원에게 메시지 프로...
1      test_1  #Person1#은 #Person2#에게 대중교통을 이용하라고 조언하고, #Pers...
2      test_2  #Person1#은 케이트에게 마샤와 히어로가 이혼하려고 한다고 말한다. 케이트는 ...
3      test_3  #Person1#은 생일을 축하하기 위해 브라이언을 위해 파티를 열고, 브라이언은 ...
4      test_4  #Person1#과 #Person2#는 올림픽 공원에 있습니다. #Person2#는...
..        ...                                                ...
494  test_495                    잭은 찰리에게 집에 와서 비디오 게임을 하자고 초대한다.
495  test_496  #Person2#는 #Person1#에게 컨트리 음악에 관심을 가지게 된 이유, 라...
496  test_497  #Person1#은 기계를 사용하는 방법을 알고 싶어합니다. 앨리스는 #Person...
497  test_498  스티브는 매튜에게 그의 계약이 끝나고 새 집을 찾고 있다고 말한다. 매튜는 이웃인 ...
498  test_499  프랭크는 승진했다. 그는 친구들을 위한 큰 파티를 열려고 한다. 벳시는 파티에 참석...

[499 rows x 2 columns]


In [14]:
model.save_pretrained("./llama3_runs/Llama-3-Open-Ko-8B-DialogueSum")